# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd  #To Import data
import numpy as np   #For Numerical analysis
import matplotlib.pyplot as plt #Data plotting and visualization
import seaborn as sns #For Data visualization


import re
import nltk
nltk.download('averaged_perceptron_tagger')
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Arjun Achuthan\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
from sqlalchemy import create_engine


engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM DisasterResponseTable", engine)
df = df[df['related'] != 2]
X = df['message']
y = df.iloc[:, 4:]

    
category_names = list(df.columns[4:])


In [3]:
df['genre'].value_counts()
df['related'].value_counts()


1    19906
0     6122
Name: related, dtype: int64

### 2. Write a tokenization function to process your text data

In [4]:
from nltk.tokenize import word_tokenize

#Regex to detect URL
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

#Function to tokenize the text
def tokenize(text):
    detected_urls = re.findall(url_regex , text)
    
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
        
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
        
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:

pipeline = Pipeline([
    ('vect' , CountVectorizer(tokenizer=tokenize)),
    ('tfidf' , TfidfTransformer()),
    ('clf' , MultiOutputClassifier(RandomForestClassifier()))
    
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
from sklearn.model_selection import train_test_split

# load data
#X, y = load_data()

# perform train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#Train classifier
pipeline.fit(X_train , y_train)


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000002287BCD3160>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [7]:

#Predict on test data
y_pred = pipeline.predict(X_test)
y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [8]:
X_train[0:1]

24210    Many anticipated that it would generate large ...
Name: message, dtype: object

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_test['related'].value_counts()

1    6547
0    2043
Name: related, dtype: int64

In [10]:
#classification report

from sklearn.metrics import classification_report, accuracy_score


# Calculate the accuracy for each of them.
for i in range(len(category_names)):
    print('Category: {} '.format(category_names[i]))
    print(classification_report(y_test.iloc[:, i].values, y_pred[:, i]))
    print('Accuracy {}\n\n'.format(accuracy_score(y_test.iloc[:, i].values, y_pred[:, i])))


Category: related 
              precision    recall  f1-score   support

           0       0.75      0.26      0.38      2043
           1       0.81      0.97      0.88      6547

    accuracy                           0.80      8590
   macro avg       0.78      0.62      0.63      8590
weighted avg       0.79      0.80      0.76      8590

Accuracy 0.8030267753201397


Category: request 
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      7126
           1       0.89      0.45      0.59      1464

    accuracy                           0.90      8590
   macro avg       0.89      0.72      0.77      8590
weighted avg       0.89      0.90      0.88      8590

Accuracy 0.8958090803259604


Category: offer 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8553
           1       0.00      0.00      0.00        37

    accuracy                           1.00      8590
   macro avg 

C:\Users\Arjun Achuthan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Accuracy 1.0


Category: water 
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      8026
           1       0.92      0.27      0.41       564

    accuracy                           0.95      8590
   macro avg       0.94      0.63      0.69      8590
weighted avg       0.95      0.95      0.94      8590

Accuracy 0.950291036088475


Category: food 
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      7622
           1       0.87      0.39      0.54       968

    accuracy                           0.93      8590
   macro avg       0.90      0.69      0.75      8590
weighted avg       0.92      0.93      0.91      8590

Accuracy 0.9250291036088475


Category: shelter 
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      7810
           1       0.85      0.26      0.39       780

    accuracy                           0.93      8590
 

Accuracy 0.859022118742724




In [11]:
# Print classification report on test data
print(classification_report(y_test.iloc[:, 0:].values, np.array([x[0:] for x in y_pred]), target_names = category_names))
print('Accuracy {}\n\n'.format(accuracy_score(y_test.iloc[:, i].values, y_pred[:, i])))

                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      6547
               request       0.89      0.45      0.59      1464
                 offer       0.00      0.00      0.00        37
           aid_related       0.80      0.61      0.69      3615
          medical_help       0.68      0.04      0.08       701
      medical_products       0.81      0.06      0.11       441
     search_and_rescue       0.62      0.02      0.04       264
              security       0.50      0.01      0.01       167
              military       0.67      0.03      0.05       289
           child_alone       0.00      0.00      0.00         0
                 water       0.92      0.27      0.41       564
                  food       0.87      0.39      0.54       968
               shelter       0.85      0.26      0.39       780
              clothing       0.82      0.07      0.13       129
                 money       0.85      

C:\Users\Arjun Achuthan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Arjun Achuthan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Arjun Achuthan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000002287BCD3160>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x000002287BCD3160>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
parameters = {
    'clf__estimator__n_estimators': [10, 20, 40]

}

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
cv.fit(X_train , y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000002287BCD3160>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [10, 20, 40]})

In [15]:
cv.best_estimator_

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000002287BCD3160>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=40)))])

In [16]:
cv.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__vect', 'estimator__tfidf', 'estimator__clf', 'estimator__vect__analyzer', 'estimator__vect__binary', 'estimator__vect__decode_error', 'estimator__vect__dtype', 'estimator__vect__encoding', 'estimator__vect__input', 'estimator__vect__lowercase', 'estimator__vect__max_df', 'estimator__vect__max_features', 'estimator__vect__min_df', 'estimator__vect__ngram_range', 'estimator__vect__preprocessor', 'estimator__vect__stop_words', 'estimator__vect__strip_accents', 'estimator__vect__token_pattern', 'estimator__vect__tokenizer', 'estimator__vect__vocabulary', 'estimator__tfidf__norm', 'estimator__tfidf__smooth_idf', 'estimator__tfidf__sublinear_tf', 'estimator__tfidf__use_idf', 'estimator__clf__estimator__bootstrap', 'estimator__clf__estimator__ccp_alpha', 'estimator__clf__estimator__class_weight', 'estimator__clf__estimator__criterion', 'estimator__clf__estimator__max_depth', 'estimator__

In [17]:

#Predict on test data
y_pred = cv.predict(X_test)

In [18]:
#classification report

from sklearn.metrics import classification_report, accuracy_score


# Calculate the accuracy for each of them.
for i in range(len(category_names)):
    print('Category: {} '.format(category_names[i]))
    print(classification_report(y_test.iloc[:, i].values, y_pred[:, i]))
    print('Accuracy {}\n\n'.format(accuracy_score(y_test.iloc[:, i].values, y_pred[:, i])))


Category: related 
              precision    recall  f1-score   support

           0       0.73      0.28      0.40      2043
           1       0.81      0.97      0.88      6547

    accuracy                           0.80      8590
   macro avg       0.77      0.62      0.64      8590
weighted avg       0.79      0.80      0.77      8590

Accuracy 0.8043073341094296


Category: request 
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      7126
           1       0.88      0.44      0.59      1464

    accuracy                           0.89      8590
   macro avg       0.89      0.71      0.76      8590
weighted avg       0.89      0.89      0.88      8590

Accuracy 0.8942956926658906


Category: offer 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8553
           1       0.00      0.00      0.00        37

    accuracy                           1.00      8590
   macro avg 

C:\Users\Arjun Achuthan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Accuracy 1.0


Category: water 
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      8026
           1       0.89      0.28      0.43       564

    accuracy                           0.95      8590
   macro avg       0.92      0.64      0.70      8590
weighted avg       0.95      0.95      0.94      8590

Accuracy 0.9505238649592549


Category: food 
              precision    recall  f1-score   support

           0       0.94      0.99      0.96      7622
           1       0.84      0.48      0.61       968

    accuracy                           0.93      8590
   macro avg       0.89      0.73      0.78      8590
weighted avg       0.93      0.93      0.92      8590

Accuracy 0.9307334109429569


Category: shelter 
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      7810
           1       0.87      0.24      0.37       780

    accuracy                           0.93      8590


In [19]:
# Print classification report on test data
print(classification_report(y_test.iloc[:, 0:].values, np.array([x[0:] for x in y_pred]), target_names = category_names))
print('Accuracy {}\n\n'.format(accuracy_score(y_test.iloc[:, i].values, y_pred[:, i])))

                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      6547
               request       0.88      0.44      0.59      1464
                 offer       0.00      0.00      0.00        37
           aid_related       0.79      0.58      0.67      3615
          medical_help       0.78      0.04      0.07       701
      medical_products       0.82      0.05      0.10       441
     search_and_rescue       0.64      0.03      0.06       264
              security       0.50      0.01      0.01       167
              military       0.71      0.04      0.08       289
           child_alone       0.00      0.00      0.00         0
                 water       0.89      0.28      0.43       564
                  food       0.84      0.48      0.61       968
               shelter       0.87      0.24      0.37       780
              clothing       0.63      0.15      0.24       129
                 money       0.80      

C:\Users\Arjun Achuthan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Arjun Achuthan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Arjun Achuthan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [20]:
# Build a custom transformer which will extract the starting verb of a sentence
from sklearn.base import BaseEstimator,TransformerMixin

class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    
    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [21]:

pipeline_improved = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize))
            , ('tfidf', TfidfTransformer())]))
        , ('starting_verb', StartingVerbExtractor())]))
    , ('clf', RandomForestClassifier())])

#pipeline_improved.get_params()

In [22]:

# Create the parameter grid based on the results of random search 
parameters_improved = {
    'clf__n_estimators': [10, 20, 40],
    'clf__criterion': ['entropy', 'gini']
    }

In [23]:
cv_improved = GridSearchCV(pipeline_improved, param_grid=parameters_improved , scoring='f1_micro')
#cv_improved.get_params()

In [24]:
cv_improved.fit(X_train , y_train)

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x000002287BCD3160>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())])),
                                                                       ('starting_verb',
                                                                        StartingVerbExtractor())])),
                                       ('clf', RandomForestClassifier())]),
             param_grid={'clf__criterion': ['entropy', 'gini'],
                         'clf__n_estimators': [10, 20, 40]},


In [25]:
#cv_improved.get_params().keys()

In [26]:

#Predict on test data
y_pred = cv_improved.predict(X_test)

In [27]:
#classification report

from sklearn.metrics import classification_report, accuracy_score


# Calculate the accuracy for each of them.
for i in range(len(category_names)):
    print('Category: {} '.format(category_names[i]))
    print(classification_report(y_test.iloc[:, i].values, y_pred[:, i]))
    print('Accuracy {}\n\n'.format(accuracy_score(y_test.iloc[:, i].values, y_pred[:, i])))

Category: related 
              precision    recall  f1-score   support

           0       0.70      0.40      0.51      2043
           1       0.83      0.95      0.89      6547

    accuracy                           0.82      8590
   macro avg       0.77      0.67      0.70      8590
weighted avg       0.80      0.82      0.80      8590

Accuracy 0.8167636786961583


Category: request 
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      7126
           1       0.88      0.43      0.58      1464

    accuracy                           0.89      8590
   macro avg       0.89      0.71      0.76      8590
weighted avg       0.89      0.89      0.88      8590

Accuracy 0.8925494761350408


Category: offer 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8553
           1       0.00      0.00      0.00        37

    accuracy                           1.00      8590
   macro avg 

C:\Users\Arjun Achuthan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy 1.0


Category: water 
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      8026
           1       0.93      0.14      0.25       564

    accuracy                           0.94      8590
   macro avg       0.94      0.57      0.61      8590
weighted avg       0.94      0.94      0.92      8590

Accuracy 0.9430733410942957


Category: food 
              precision    recall  f1-score   support

           0       0.92      1.00      0.95      7622
           1       0.89      0.27      0.42       968

    accuracy                           0.91      8590
   macro avg       0.90      0.64      0.69      8590
weighted avg       0.91      0.91      0.89      8590

Accuracy 0.9145518044237485


Category: shelter 
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      7810
           1       0.93      0.07      0.13       780

    accuracy                           0.92      8590
 


Accuracy 0.9887077997671712


Category: earthquake 
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      7782
           1       0.92      0.49      0.64       808

    accuracy                           0.95      8590
   macro avg       0.93      0.74      0.81      8590
weighted avg       0.95      0.95      0.94      8590

Accuracy 0.9481955762514552


Category: cold 
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      8412
           1       1.00      0.01      0.01       178

    accuracy                           0.98      8590
   macro avg       0.99      0.50      0.50      8590
weighted avg       0.98      0.98      0.97      8590

Accuracy 0.9793946449359721


Category: other_weather 
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      8159
           1       0.75      0.01      0.01       431

    accuracy                

In [28]:
# Print classification report on test data
print(classification_report(y_test.iloc[:, 0:].values, np.array([x[0:] for x in y_pred]), target_names = category_names))
print('Accuracy {}\n\n'.format(accuracy_score(y_test.iloc[:, i].values, y_pred[:, i])))

                        precision    recall  f1-score   support

               related       0.83      0.95      0.89      6547
               request       0.88      0.43      0.58      1464
                 offer       0.00      0.00      0.00        37
           aid_related       0.85      0.47      0.61      3615
          medical_help       0.71      0.01      0.01       701
      medical_products       0.83      0.01      0.02       441
     search_and_rescue       0.50      0.00      0.01       264
              security       1.00      0.01      0.01       167
              military       0.00      0.00      0.00       289
           child_alone       0.00      0.00      0.00         0
                 water       0.93      0.14      0.25       564
                  food       0.89      0.27      0.42       968
               shelter       0.93      0.07      0.13       780
              clothing       0.75      0.05      0.09       129
                 money       0.67      

C:\Users\Arjun Achuthan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Arjun Achuthan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Arjun Achuthan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 9. Export your model as a pickle file

In [29]:
import pickle
pickle.dump(cv, open('model.pkl', 'wb'))

In [30]:
pickle.dump(cv, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [31]:
cv_improved.best_estimator_

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x000002287BCD3160>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('starting_verb',
                                                 StartingVerbExtractor())])),
                ('clf', RandomForestClassifier(n_estimators=40))])